In [1]:
from math import sqrt
import pandas as pd
import numpy as np

from pathlib import Path
from collections import defaultdict
import pickle
import gc

from tqdm import tqdm
from pandarallel import pandarallel

from scipy.sparse import csr_matrix, coo_matrix
import implicit
import catboost

import sys
sys.path.append("..")
from src.utils import *
from src.dataset import *
from src.trending import *
from src.level_1 import *

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from catboost import Pool, CatBoostClassifier, cv
from catboost.utils import get_roc_curve, create_cd
from catboost.eval.catboost_evaluation import CatboostEvaluation

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# TRAIN 

In [2]:
SEED = 1
N = 12
TEST_ON = 1

cv_iteration = 0

dataset = Dataset(skip_days=7 * cv_iteration, test_days=7 * TEST_ON)
train, test = dataset.get_train_and_test()
articles = dataset.get_articles()
customers = dataset.get_customers()

train = add_quotient(train=train)
print("Dataset created")

/home/rv/projects/HM_Personalized_Fashion_Recomendations/new_notebooks/../src/trending.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ldbw'] = df['t_dat'].parallel_apply(lambda d: last_ts - (last_ts - d).floor('7D'))


Dataset created


In [3]:
purch_data = get_purchase_data(train)
purch_data.to_csv("../tmp/purchase_data_train.csv", index=False)

similar_purch_data = get_similar_data(purch_data, train, articles, customers, 
                                      min_w1_count_for_actual_article=25, similar_count_for_article=3)
similar_purch_data.to_csv("../tmp/als_similarity_train.csv", index=False)

purch_similar = purch_data.merge(similar_purch_data, on=["customer_id", "article_id"], how="outer")
purch_similar = purch_similar.fillna(-1).astype(np.int32)

actual_articles = get_actual_articles(train, min_count=25, days=3)
purch_similar = purch_similar[purch_similar["article_id"].isin(actual_articles)]

del purch_data, similar_purch_data
gc.collect()

purch_similar.to_csv("../tmp/purch_similar_train.csv", index=False)
purch_similar

100%|██████████| 27101148/27101148 [00:40<00:00, 667480.76it/s]


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 105542/105542 [00:40<00:00, 2585.16it/s]


,customer_id,article_id,purchase_score,similar_parent_purchase_score,als_similarity
1,0,16003,330,330,212
2,0,16023,28864,28864,98
45,1,42626,0,0,528
95,2,78503,146809,146809,113
109,4,60763,1681,1681,169
...,...,...,...,...,...
105328285,1367957,103973,-1,117225,13
105328286,1371062,85256,-1,0,26
105328287,1371208,102422,-1,0,17
105328289,1371341,60808,-1,0,29


In [4]:
general_count_popular = get_general_count_popular(train, customers, N=12)
group_count_popular = get_group_count_popular(train, customers, N=12)

popular_data = general_count_popular.merge(group_count_popular, on=["customer_id", "article_id"], how='outer')
popular_data = popular_data.fillna(-1).astype(np.int32)

del general_count_popular, group_count_popular
gc.collect()

popular_data.to_csv("../tmp/popular_train.csv", index=False)
popular_data

,customer_id,article_id,general_popular_count,general_popular_count_rank,group_popular_count,group_popular_count_rank
0,0,103108,1306,0,138,3
1,0,95217,844,1,-1,-1
2,0,104072,819,2,141,2
3,0,104553,718,3,104,11
4,0,3091,716,4,104,9
...,...,...,...,...,...,...
22624847,1371979,67543,-1,-1,131,2
22624848,1371979,103304,-1,-1,116,4
22624849,1371979,103033,-1,-1,100,7
22624850,1371979,103303,-1,-1,87,8


In [5]:
purch_similar_popular = purch_similar.merge(popular_data, on=["customer_id", "article_id"], how="outer")
purch_similar_popular = purch_similar_popular.fillna(-1).astype(np.int32)

del purch_similar, popular_data
gc.collect()

purch_similar_popular.to_csv("../tmp/purch_similar_popular_train.csv", index=False)
purch_similar_popular

,customer_id,article_id,purchase_score,similar_parent_purchase_score,als_similarity,general_popular_count,general_popular_count_rank,group_popular_count,group_popular_count_rank
0,0,16003,330,330,212,-1,-1,-1,-1
1,0,16023,28864,28864,98,-1,-1,-1,-1
2,1,42626,0,0,528,-1,-1,-1,-1
3,2,78503,146809,146809,113,-1,-1,-1,-1
4,4,60763,1681,1681,169,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
56298697,1371979,67543,-1,-1,-1,-1,-1,131,2
56298698,1371979,103304,-1,-1,-1,-1,-1,116,4
56298699,1371979,103033,-1,-1,-1,-1,-1,100,7
56298700,1371979,103303,-1,-1,-1,-1,-1,87,8


# TEST 

In [2]:
SEED = 1
N = 12
TEST_ON = 0

cv_iteration = 0

dataset = Dataset(skip_days=7 * cv_iteration, test_days=7 * TEST_ON)
train, test = dataset.get_train_and_test()
articles = dataset.get_articles()
customers = dataset.get_customers()

train = add_quotient(train=train)
print("Dataset created")

/home/rv/projects/HM_Personalized_Fashion_Recomendations/new_notebooks/../src/trending.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ldbw'] = df['t_dat'].parallel_apply(lambda d: last_ts - (last_ts - d).floor('7D'))


Dataset created


In [ ]:
purch_data = get_purchase_data(train)
purch_data.to_csv("../tmp/purchase_data_test.csv", index=False)

similar_purch_data = get_similar_data(purch_data, train, articles, customers, 
                                      min_w1_count_for_actual_article=25, similar_count_for_article=3)
similar_purch_data.to_csv("../tmp/als_similarity_test.csv", index=False)

purch_similar = purch_data.merge(similar_purch_data, on=["customer_id", "article_id"], how="outer")
purch_similar = purch_similar.fillna(-1).astype(np.int32)

actual_articles = get_actual_articles(train, min_count=25, days=3)
purch_similar = purch_similar[purch_similar["article_id"].isin(actual_articles)]

del purch_data, similar_purch_data
gc.collect()

purch_similar.to_csv("../tmp/purch_similar_test.csv", index=False)
purch_similar

100%|██████████| 27306439/27306439 [00:24<00:00, 1133198.78it/s]


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 105542/105542 [00:32<00:00, 3270.10it/s]


In [ ]:
general_count_popular = get_general_count_popular(train, customers, N=12)
group_count_popular = get_group_count_popular(train, customers, N=12)

popular_data = general_count_popular.merge(group_count_popular, on=["customer_id", "article_id"], how='outer')
popular_data = popular_data.fillna(-1).astype(np.int32)

del general_count_popular, group_count_popular
gc.collect()

popular_data.to_csv("../tmp/popular_test.csv", index=False)
popular_data

In [ ]:
purch_similar_popular = purch_similar.merge(popular_data, on=["customer_id", "article_id"], how="outer")
purch_similar_popular = purch_similar_popular.fillna(-1).astype(np.int32)

del purch_similar, popular_data
gc.collect()

purch_similar_popular.to_csv("../tmp/purch_similar_popular_test.csv", index=False)
purch_similar_popular